# 异步协程语法

从ES8开始ECMAScript正式支持异步协程语法,使用关键字`async/await`来声明异步,其语义和用法与python中的一致.Promise作用相当于python中的Future.

本质来说`async/await`是一种用于处理JS异步操作的语法糖,可以帮助我们利用同步语法使用Promise,从而编写更加优雅的代码.

不过js的协程语法不用配合显式的声明事件循环.js的事件循环是隐式的.以异步访问为例:

In [1]:
import fetch from 'node-fetch'
async function testAsync(){
    try{
        let response = await fetch(
            'http://www.baidu.com',
            {
                method: 'GET', 
            }
        )
        if (response.ok){
            let content = await response.text()
            console.log(content.slice(0,100))
        }else{
            console.error("http code error")
        }
    }catch(e){
       console.error(error)
    }
}
testAsync()

<!DOCTYPE html><!--STATUS OK--><html><head><meta http-equiv="Content-Type" content="text/html;charse


## 构造Promise用于异步协程语法

更多的时候遗留代码是回调的形式,这自然无法用于协程语法,下面我们来介绍常用的构造Promise的方法.我们使用文件阅读的接口作为例子.这个接口非常典型.

In [2]:
import fs from "fs"

### 从callback构造Promise

fs的`readFile`接口就是一个典型的基于回调的非阻塞接口.我们来将其改造为Promise.

这种改造的基本思路是将使用回调的接口调用步骤放在Promise的构造函数中执行,在回调函数里将正确的数据使用`resolve`函数提交到Promise对象的值中.

In [3]:
function readFileFromCallback(path,encoding='utf-8'){
    return new Promise((resolve, reject)=>{
        fs.readFile(
            path, 
            encoding,
            function (err, data) {
              if (err) {
                  reject(err)
              }
              resolve(data)
            }
        )
    })
}

In [4]:
async function readREADMEFromCallback(){
    try{
        let data = await readFileFromCallback('./README.md')
        let msg = data.slice(0,20)
        console.log(`get msg: ${msg}`)
    }catch(e){
       console.error(`get error: ${e}`)
    }
    try{
        let data = await readFileFromCallback('./README5.md')
        let msg = data.slice(0,20)
        console.log(`get msg: ${msg}`)
    }catch(e){
       console.error(`get error: ${e}`)
    }
}
readREADMEFromCallback()

get msg: # Javascript基础语法

ES


get error: Error: ENOENT: no such file or directory, open './README5.md'


### 从EventEmitter构造Promise

要从`EventEmitter`中构造Promise只能使用`once`方法.一种通用的方式是在结束事件的回调函数中使用`once`包裹`resolve`

In [5]:
function readFileFromEmitter(path,encoding='utf-8'){
    return new Promise((resolve, reject)=>{
        let res = ""
        let file = fs.createReadStream(path, encoding)
        file.on("data",(data)=>{
            res += data
        }).once("end",()=>resolve(res))
        .once("error",(err)=>{
            file.destroy(err)
            reject(err)
        })
    })
}

In [6]:
async function readREADMEFromEmitter(){
    try{
        let data = await readFileFromEmitter('./README.md')
        let msg = data.slice(0,20)
        console.log(`get msg: ${msg}`)
    }catch(e){
       console.error(`get error: ${e}`)
    }
    try{
        let data = await readFileFromEmitter('./README.md5')
        let msg = data.slice(0,20)
        console.log(`get msg: ${msg}`)
    }catch(e){
       console.error(`get error: ${e}`)
    }
}
readREADMEFromEmitter()

get msg: # Javascript基础语法

ES


Error: ENOENT: no such file or directory, open './README.md5'get error: Error: ENOENT: no such file or directory, open './README.md5'


如果你使用的是node而且刚好node版本高于11.13.0,那么我们可以使用`events.once`来构造,这个方法会自动检测`error`事件,

In [7]:
import {once} from 'events'
async function readREADMEFromEmitter2(){
    try{
        let [data] = await once(fs.createReadStream('./README.md', 'utf-8'),"data")
        let msg = data.slice(0,20)
        console.log(`get msg: ${msg}`)
    }catch(e){
       console.error(`get error: ${e}`)
    }
    
    try{
        let [data] = await once(fs.createReadStream('./README.md5', 'utf-8'),"data")
        let msg = data.slice(0,20)
        console.log(`get msg: ${msg}`)
    }catch(e){
       console.error(`get error: ${e}`)
    }
}
readREADMEFromEmitter2()

get msg: # Javascript基础语法

ES


get error: Error: ENOENT: no such file or directory, open './README.md5'


## 异步迭代器

从ES9开始ECMAScript正式支持异步迭代器语法,功能上讲和python的异步迭代器一致,但语法不同,js中使用语法

```js
for await( let x of AsyncIterator) {
      ...
}
```

其中`AsyncIterator`就是异步迭代器,只要迭代器的next方法返回的是一个Promise实例就是异步迭代器.

In [1]:
let promises = [1,2,3,4,5].map(i=>Promise.resolve(i))

In [4]:
(async ()=>{
    for await (let p of promises){
        console.log(p)
    }
})()

1
2
3
4
5


In [1]:
import { Readable } from 'stream'
class Timer extends Readable {
    constructor(options) {
        super(options)
    }
    _read() {
        setInterval(()=>{
            let time  = new Date()
            this.push(time.toString(),"utf-8")
        },1000)
    }
}


async function a(){
    let timer = new Timer({encoding:"utf-8"})
    for await (let d of timer){
        console.log(d)
    }
}

In [ ]:
a()

Wed Feb 26 2020 21:43:18 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:18 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:19 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:19 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:19 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:19 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:20 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 2020 21:43:21 GMT+0800 (中国标准时间)
Wed Feb 26 